In [1]:
import pandas as pd
import re
import numpy as np
import copy

In [2]:
times = pd.read_csv('~/Scrivania/Uni/LM/FoCS/Esame/timesData.csv')
shan = pd.read_csv("~/Scrivania/Uni/LM/FoCS/Esame/shanghaiData.csv")
cwur = pd.read_csv("~/Scrivania/Uni/LM/FoCS/Esame/cwurData.csv")
eas = pd.read_csv("~/Scrivania/Uni/LM/FoCS/Esame/educational_attainment_supplementary_data.csv")

In [3]:
cwur.rename(columns={"institution":"university_name"},inplace=True)

Vado a verificare se qualche università è scitta in modo diverso nelle varie classifiche. Per prima cosa creo l'elenco delle università e poi vado a considerare le università che sono in times e non in shan.

In [4]:
times_uni = times['university_name'].drop_duplicates()
shan_uni = shan['university_name'].drop_duplicates()

In [5]:
print(f'times: {len(times_uni)}, shan: {len(shan_uni)}')

times: 818, shan: 659


In [6]:
p = pd.merge(shan, times, on='university_name', how='left', suffixes=('_shan', '_times'))

In [7]:
uni_not_in_times = p[p['world_rank_times'].isnull()]['university_name'].drop_duplicates()

Noto che alcune università conengono una sigla. Vado a vedere se eliminando la sigla queste sono presenti in times.

In [8]:
a = uni_not_in_times[uni_not_in_times.str.contains('^.+\(.+\)$') == True]
a

24             Massachusetts Institute of Technology (MIT)
249                     University of Paris Sud (Paris 11)
645                 Joseph Fourier University (Grenoble 1)
755                University of Paris Descartes (Paris 5)
756                  University of Paris Diderot (Paris 7)
872                  Paul Sabatier University (Toulouse 3)
1114                   Henri Poincare University (Nancy 1)
1263                University of Paris Dauphine (Paris 9)
1278     University of the Mediterranean (Aix-Marseille 2)
1493              University of Provence (Aix-Marseille 1)
10416              Karlsruhe Institute of Technology (KIT)
13367    Western University (The University of Western ...
13998                   University of Paris-Sud (Paris 11)
14269                  Université libre de Bruxelles (ULB)
14698                     Vrije Universiteit Brussel (VUB)
15069                  Paris Dauphine University (Paris 9)
15120    University at Albany (State University of New .

In [9]:
universita = a.str.replace('\s?\(.+\)$', '')

<ipython-input-9-3ee221737bee>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  universita = a.str.replace('\s?\(.+\)$', '')


In [10]:
for uni in universita:
    print(times_uni[times_uni == uni])

2    Massachusetts Institute of Technology
Name: university_name, dtype: object
Series([], Name: university_name, dtype: object)
781    Joseph Fourier University
Name: university_name, dtype: object
Series([], Name: university_name, dtype: object)
Series([], Name: university_name, dtype: object)
Series([], Name: university_name, dtype: object)
Series([], Name: university_name, dtype: object)
Series([], Name: university_name, dtype: object)
Series([], Name: university_name, dtype: object)
Series([], Name: university_name, dtype: object)
186    Karlsruhe Institute of Technology
Name: university_name, dtype: object
Series([], Name: university_name, dtype: object)
Series([], Name: university_name, dtype: object)
Series([], Name: university_name, dtype: object)
548    Vrije Universiteit Brussel
Name: university_name, dtype: object
972    Paris Dauphine University
Name: university_name, dtype: object
Series([], Name: university_name, dtype: object)


Alcune università come 'Massachusetts Institute of Technology' sono presenti in times, quindi eimino tutte le sigle da shan.

In [11]:
shan['university_name'] = shan['university_name'].str.replace('\s?\(.+\)$', '')

<ipython-input-11-97c9fbc0bbfc>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  shan['university_name'] = shan['university_name'].str.replace('\s?\(.+\)$', '')


Analogamnete a quanto fatto prima elimino le sigle anche da times e cwur per evitare una situazione come quella descritta precedentemente. In questo modo abbiamo risolto il problema il problema delle sigle.

In [12]:
times['university_name'] = times['university_name'].str.replace('\s?\(.+\)$', '')

<ipython-input-12-79f8b15de512>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  times['university_name'] = times['university_name'].str.replace('\s?\(.+\)$', '')


In [13]:
cwur['university_name'] = cwur['university_name'].str.replace('\s?\(.+\)$', '')

<ipython-input-13-2f5e76ac5643>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  cwur['university_name'] = cwur['university_name'].str.replace('\s?\(.+\)$', '')


## 1. For each university, extract from the times dataset the most recent and the least recent data, obtaining two separate dataframes

In [14]:
times.sort_values("year",ascending = False)
#Correggo errori di battitura notati risolvendo il punto 5
times['country'] = times['country'].str.replace('Unted Kingdom', 'United Kingdom')
times['country'] = times['country'].str.replace('Unisted States of America', 'United States of America')

In [15]:
times_most_recent = times.loc[times.groupby('university_name').idxmax()['year']]
times_least_recent = times.loc[times.groupby('university_name').idxmin()['year']]
times_most_recent

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405.0,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016
2003.0,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,"17,422",15.9,15%,48 : 52,2016
2056.0,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,"16,099",24.2,17%,32 : 68,2016
1908.0,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14%,54 : 46,2016
2105.0,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,"9,252",19.2,18%,48 : 52,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1856.0,54,École Normale Supérieure,France,70.6,85.5,47.7,87.1,37.1,69.0,"2,400",7.9,20%,46 : 54,2016
2013.0,201-250,École Normale Supérieure de Lyon,France,41.6,65.6,30.0,69.0,31.7,-,"2,218",8.0,14%,49 : 51,2016
1904.0,=101,École Polytechnique,France,53.5,92.8,44.6,64.7,82.3,57.9,"2,429",4.8,30%,18 : 82,2016
1833.0,31,École Polytechnique Fédérale de Lausanne,Switzerland,61.3,98.6,67.5,94.6,65.4,76.1,"9,666",10.5,54%,27 : 73,2016


## 2. For each university, compute the improvement in income between the least recent and the most recent data points

Per andare a calcolare la differenza tra gli income per prima cosa bisogna trasformare l'attributo 'income' da object ad attributo numerico. Una volta fatta la differenza abbiamo eliminato i valori nulli che corrispondono alle università che in uno dei due dataset (o in entrambi) non avevano il valore dell'attributo.

In [16]:
times_most_recent['income'] = pd.to_numeric(times_most_recent['income'], errors = 'coerce') 
times_least_recent['income'] = pd.to_numeric(times_least_recent['income'], errors = 'coerce') 

In [17]:
times_most_recent.set_index('university_name', inplace = True)
times_least_recent.set_index('university_name', inplace = True)

In [18]:
income_diff = times_most_recent['income'] - times_least_recent['income']
income_diff = income_diff.reset_index()
income_diff.columns = ['university_name', 'difference']
income_diff.dropna()

,university_name,difference
1,Aalborg University,7.3
2,Aalto University,-0.3
3,Aarhus University,6.8
4,Aberystwyth University,-4.2
5,Adam Mickiewicz University,0.0
...,...,...
812,Zhejiang University,25.9
813,École Normale Supérieure,6.4
814,École Normale Supérieure de Lyon,5.6
816,École Polytechnique Fédérale de Lausanne,27.4


## 3. Find the university with the largest increase computed in the previous point

In [19]:
income_diff[income_diff['difference'] == income_diff['difference'].max()]

,university_name,difference
428,TU Dresden,67.8


## 4. For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for Aarhus University the value is 122-73=49). Notice that some rankings are expressed as a range

Prima di tutto bisogna notare che alcuni valori dei rank nelle classifiche non sono espressi con un valore numerico, ma sono in un intervallo. Inoltre, nel dataset times alcune università sono classificate parimerito e la posizione è aticipata da "=". Per le università classificate con un inetrvallo abbiamo considerato la media e inoltre abbiamo semplicemnete eliminato l'uguale. 

In [20]:
def pulisci(num):
    re_comp1 = re.compile("^(\d+)\-+(\d*)")
    re_comp2 = re.compile("^\=(\d*)$")
    m = re_comp1.search(num)
    m2 = re_comp2.search(num)
    if m:
        a = (int(m.group(1)) + int(m.group(2))) / 2
        return int(a)
    elif m2:
        a = m2.group(1)
        return int(a)
    return int(num)

In [21]:
times_most_recent['world_rank'] = times_most_recent['world_rank'].apply(pulisci)

In [22]:
times_most_recent

,world_rank,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
university_name,,,,,,,,,,,,,
AGH University of Science and Technology,700,Poland,14.2,17.9,3.7,35.7,NaN,-,"35,569",17.0,1%,-,2016
Aalborg University,225,Denmark,25.1,71.0,28.4,73.8,43.7,-,"17,422",15.9,15%,48 : 52,2016
Aalto University,275,Finland,31.1,65.4,32.8,62.1,61.6,-,"16,099",24.2,17%,32 : 68,2016
Aarhus University,106,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14%,54 : 46,2016
Aberystwyth University,325,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,"9,252",19.2,18%,48 : 52,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...
École Normale Supérieure,54,France,70.6,85.5,47.7,87.1,37.1,69.0,"2,400",7.9,20%,46 : 54,2016
École Normale Supérieure de Lyon,225,France,41.6,65.6,30.0,69.0,31.7,-,"2,218",8.0,14%,49 : 51,2016
École Polytechnique,101,France,53.5,92.8,44.6,64.7,82.3,57.9,"2,429",4.8,30%,18 : 82,2016


In [23]:
shan_most_recent = shan.iloc[shan.groupby("university_name")["year"].idxmax()]
shan_most_recent

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year
4697,301-400,Aalborg University,5,NaN,0.0,0.0,11.2,4.6,30.4,16.8,2015
4797,401-500,Aalto University,4-6,NaN,0.0,0.0,0.0,10.4,33.4,17.0,2015
4469,73,Aarhus University,2,27.3,11.5,22.1,12.3,25.8,51.8,31.0,2015
4497,101-150,Aix Marseille University,5-6,NaN,13.6,0.0,15.2,20.3,50.1,27.6,2015
3115,102-150,Aix-Marseille University,4-6,NaN,15.6,0.0,16.1,22.4,44.3,15.3,2011
...,...,...,...,...,...,...,...,...,...,...,...
3512,401-500,Yamaguchi University,17-23,NaN,0.0,0.0,16.1,2.8,20.0,13.2,2011
4695,201-300,Yeshiva University,79-102,NaN,0.0,0.0,6.3,21.7,31.5,21.6,2015
4696,201-300,Yonsei University,2-5,NaN,0.0,0.0,8.6,4.9,52.1,21.4,2015
4396,401-500,York University,19-21,NaN,0.0,0.0,0.0,10.1,30.6,12.4,2014


In [24]:
shan_most_recent["world_rank"] = shan_most_recent["world_rank"].apply(pulisci)
shan_most_recent

<ipython-input-24-a975518a78d8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shan_most_recent["world_rank"] = shan_most_recent["world_rank"].apply(pulisci)


,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year
4697,350,Aalborg University,5,NaN,0.0,0.0,11.2,4.6,30.4,16.8,2015
4797,450,Aalto University,4-6,NaN,0.0,0.0,0.0,10.4,33.4,17.0,2015
4469,73,Aarhus University,2,27.3,11.5,22.1,12.3,25.8,51.8,31.0,2015
4497,125,Aix Marseille University,5-6,NaN,13.6,0.0,15.2,20.3,50.1,27.6,2015
3115,126,Aix-Marseille University,4-6,NaN,15.6,0.0,16.1,22.4,44.3,15.3,2011
...,...,...,...,...,...,...,...,...,...,...,...
3512,450,Yamaguchi University,17-23,NaN,0.0,0.0,16.1,2.8,20.0,13.2,2011
4695,250,Yeshiva University,79-102,NaN,0.0,0.0,6.3,21.7,31.5,21.6,2015
4696,250,Yonsei University,2-5,NaN,0.0,0.0,8.6,4.9,52.1,21.4,2015
4396,450,York University,19-21,NaN,0.0,0.0,0.0,10.1,30.6,12.4,2014


In [25]:
cwur_most_recent = cwur.iloc[cwur.groupby("university_name")["year"].idxmax()]
cwur_most_recent["world_rank"] = cwur_most_recent["world_rank"].apply(lambda x: int(x))

<ipython-input-25-b0279d29033c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cwur_most_recent["world_rank"] = cwur_most_recent["world_rank"].apply(lambda x: int(x))


In [26]:
cwur_most_recent

,world_rank,university_name,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
1981,782,AGH University of Science and Technology,Poland,4,267,279,218,708,891,511,896.0,398,44.26,2015
1764,565,Aalborg University,Denmark,5,367,439,218,596,660,287,520.0,401,44.59,2015
1620,421,Aalto University,Finland,4,367,224,218,336,403,645,424.0,246,45.10,2015
1321,122,Aarhus University,Denmark,2,310,165,106,91,81,101,95.0,154,49.65,2015
2013,814,Aberystwyth University,United Kingdom,59,367,567,218,842,552,511,754.0,607,44.22,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2080,881,École centrale de Lyon,France,45,367,255,218,856,841,812,991.0,570,44.15,2015
1236,37,École normale supérieure - Paris,France,2,8,502,54,379,157,645,295.0,871,58.80,2015
1920,721,École normale supérieure de Cachan,France,32,121,323,218,895,776,428,826.0,871,44.33,2015
1670,471,École normale supérieure de Lyon,France,18,367,567,135,481,210,511,413.0,589,44.87,2015


In [27]:
cwur_most_recent
shan_most_recent
times_most_recent


merged_rank =  pd.merge(cwur_most_recent,shan_most_recent, on ="university_name",how="inner",suffixes=["_cwur","_shang"])
merged_tot = pd.merge(merged_rank,times_most_recent,on = "university_name",how="inner")

In [28]:
merged_tot.columns

Index(['world_rank_cwur', 'university_name', 'country_x', 'national_rank_cwur',
       'quality_of_education', 'alumni_employment', 'quality_of_faculty',
       'publications', 'influence', 'citations_x', 'broad_impact', 'patents',
       'score', 'year_cwur', 'world_rank_shang', 'national_rank_shang',
       'total_score_x', 'alumni', 'award', 'hici', 'ns', 'pub', 'pcp',
       'year_shang', 'world_rank', 'country_y', 'teaching', 'international',
       'research', 'citations_y', 'income', 'total_score_y', 'num_students',
       'student_staff_ratio', 'international_students', 'female_male_ratio',
       'year'],
      dtype='object')

In [29]:
merged_tot_R = merged_tot[["university_name","world_rank_cwur","world_rank","world_rank_shang"]]

In [30]:
merged_tot_R

,university_name,world_rank_cwur,world_rank,world_rank_shang
0,Aalborg University,565,225,350
1,Aalto University,421,275,450
2,Aarhus University,122,106,73
3,Aix-Marseille University,206,275,126
4,Aristotle University of Thessaloniki,459,700,450
...,...,...,...,...
366,Yale University,11,12,11
367,Yeshiva University,171,164,250
368,Yonsei University,98,325,250
369,York University,337,325,450


In [31]:
merged_tot_R['max_value'] = merged_tot_R.max(axis=1)

<ipython-input-31-79c80bf920f6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_tot_R['max_value'] = merged_tot_R.max(axis=1)


In [32]:
merged_tot_R['min_value'] = merged_tot_R.min(axis=1)

<ipython-input-32-de1e982d57ad>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_tot_R['min_value'] = merged_tot_R.min(axis=1)


In [33]:
merged_tot_R["maximum difference"] = merged_tot_R["max_value"] - merged_tot_R["min_value"]

<ipython-input-33-2088d4db6ac3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_tot_R["maximum difference"] = merged_tot_R["max_value"] - merged_tot_R["min_value"]


In [34]:
merged_tot_R[["university_name","maximum difference"]]

,university_name,maximum difference
0,Aalborg University,340
1,Aalto University,175
2,Aarhus University,49
3,Aix-Marseille University,149
4,Aristotle University of Thessaloniki,250
...,...,...
366,Yale University,1
367,Yeshiva University,86
368,Yonsei University,227
369,York University,125


In [35]:
len(merged_tot_R[["university_name","maximum difference"]].groupby("university_name"))

371

## 5. Consider only the most recent data point of the times dataset. Compute the number of male and female students for each country.

In [36]:
times['num_students'] = pd.to_numeric(times['num_students'].str.replace(',', ''))

In [37]:
times_most_recent = times.loc[times.groupby('university_name').idxmax()['year']]

In [38]:
def fem_ratio(s):
    m = s.str.extract('\s*(?P<female>\d+)')
    fem = pd.to_numeric(m['female']) / 100
    return fem

In [39]:
fem = fem_ratio(times_most_recent['female_male_ratio'])
#times_most_recent['num_students']
times_most_recent['female'] = (times_most_recent['num_students'] * fem).round(0)
times_most_recent['male'] = times_most_recent['num_students'] - times_most_recent['female']

In [40]:
times_most_recent.groupby('country').sum()[['female', 'male']]

,female,male
country,,
Argentina,67191.0,41182.0
Australia,391736.0,321640.0
Austria,68364.0,66113.0
Bangladesh,21323.0,41393.0
Belarus,20219.0,9084.0
...,...,...
Uganda,18670.0,18670.0
Ukraine,17846.0,19250.0
United Arab Emirates,9516.0,4931.0


## 6. Find the universities where the ratio between female and male is below the average ratio (computed over all universities)

In [41]:
times['fem_ratio'] = fem_ratio(times['female_male_ratio'])
times['male_ratio'] = 1 - times['fem_ratio']

In [42]:
times[times['male_ratio'] == 0].shape[0]

2

In [43]:
times2 = times[times['male_ratio'] != 0]
ratio = times2['fem_ratio'] / times2['male_ratio']
avg = ratio.mean()
avg

1.0684463755952236

In [44]:
times2['ratio'] = ratio

<ipython-input-44-6f48264178b4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  times2['ratio'] = ratio


In [45]:
times3 = times2[times2['ratio'] < avg][['university_name', 'num_students', 'country', 'ratio']].drop_duplicates()
times3[['university_name', 'ratio']]

,university_name,ratio
1,California Institute of Technology,0.492537
2,Massachusetts Institute of Technology,0.587302
3,Stanford University,0.724138
4,Princeton University,0.818182
5,University of Cambridge,0.851852
...,...,...
2591,Warsaw University of Technology,0.515152
2597,Xidian University,0.408451
2598,Yeungnam University,0.923077
2599,Yıldız Technical University,0.562500


## 7. For each country compute the fraction of students that are in one of the universities computed in the previous point.

In [46]:
times4 = times[['country', 'num_students']].drop_duplicates()

In [47]:
total_students = times4.groupby('country').sum()['num_students']
total_students.reset_index()

,country,num_students
0,Argentina,108373.0
1,Australia,743627.0
2,Austria,134477.0
3,Bangladesh,62716.0
4,Belarus,29303.0
...,...,...
65,Uganda,37340.0
66,Ukraine,37096.0
67,United Arab Emirates,14447.0
68,United Kingdom,1324842.0


In [48]:
times5 = times3.groupby('country').sum()['num_students'].reset_index()
times5.head()

,country,num_students
0,Australia,146235.0
1,Austria,61033.0
2,Bangladesh,62716.0
3,Brazil,278774.0
4,Canada,86779.0


In [49]:
merged = pd.merge(times5, total_students, on='country', suffixes=('_sel', '_tot'))
merged['ratio'] = (merged['num_students_sel'] / merged['num_students_tot']).round(2)
merged.head()

,country,num_students_sel,num_students_tot,ratio
0,Australia,146235.0,743627.0,0.20
1,Austria,61033.0,134477.0,0.45
2,Bangladesh,62716.0,62716.0,1.00
3,Brazil,278774.0,534688.0,0.52
4,Canada,86779.0,717054.0,0.12


## 8. Read the file educational_attainment_supplementary_data.csv, discarding any row with missing country_name or series_name

In [50]:
eas.dropna(subset=['country_name', 'series_name'], inplace=True)

## 9. From attainment build a dataframe with the same data, but with 4 columns: country_name, series_name, year, value

In [51]:
anni = list(eas.columns[2:])

In [52]:
eas2 = eas.melt(id_vars=('country_name', 'series_name'), value_vars=anni, var_name='year',
                value_name='value').sort_values(by=['country_name', 'series_name']).dropna().reset_index(drop=True)
eas2

,country_name,series_name,year,value
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.33000
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1990,0.44000
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1995,0.57000
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2000,0.75000
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2005,0.86000
...,...,...,...,...
318285,Zimbabwe,UIS: Percentage of population age 25+ with som...,2012,13.84323
318286,Zimbabwe,UIS: Percentage of population age 25+ with som...,2012,16.68491
318287,Zimbabwe,UIS: Percentage of population age 25+ with unk...,2012,11.99412
318288,Zimbabwe,UIS: Percentage of population age 25+ with unk...,2012,5.77150


## 10. For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking).

Andiamo a creare tre data frame, ogniuno dei quali contiene le università di una delle tre classifiche e una colonna count con valore pari a 1. Successivamente faremo due join per unire questi data frame e otteniamo le università presenti almeno una volta in tutte le classifiche. A questo aggiungiamo la colonna somma che rappresenta il numero di classifiche in cui un'università è presente.

In [53]:
times2 = times['university_name'].drop_duplicates()
times2 = pd.DataFrame(times2.value_counts()).reset_index().rename(columns={"index": "university_name", "university_name": "count_times"})
times2

,university_name,count_times
0,National Taiwan University,1
1,Hunan University,1
2,National Institute of Applied Sciences of Lyon,1
3,University of Jyväskylä,1
4,Bayreuth University,1
...,...,...
813,Royal College of Surgeons in Ireland,1
814,University of Burgundy,1
815,The Open University,1
816,University of Toronto,1


In [54]:
cwur

,world_rank,university_name,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2195,996,University of the Algarve,Portugal,7,367,567,218,926,845,812,969.0,816,44.03,2015
2196,997,Alexandria University,Egypt,4,236,566,218,997,908,645,981.0,871,44.03,2015
2197,998,Federal University of Ceará,Brazil,18,367,549,218,830,823,812,975.0,824,44.03,2015
2198,999,University of A Coruña,Spain,40,367,567,218,886,974,812,975.0,651,44.02,2015


In [55]:
cwur2 = cwur["university_name"].drop_duplicates()
cwur2 = pd.DataFrame(cwur2.value_counts()).reset_index().rename(columns={"index": "university_name", "university_name": "count_cwur"})
cwur2

,university_name,count_cwur
0,National Taiwan University,1
1,Western University,1
2,North Dakota State University - Main Campus,1
3,University of Massachusetts Lowell,1
4,Rice University,1
...,...,...
1017,École Centrale Paris,1
1018,Complutense University of Madrid,1
1019,"University of North Texas, Denton",1
1020,University of New South Wales,1


In [56]:
shan2 = shan['university_name'].drop_duplicates()
shan2 = pd.DataFrame(shan2.value_counts()).reset_index().rename(columns={"index": "university_name", "university_name": "count_shan"})
shan2

,university_name,count_shan
0,University of Alaska - Fairbanks,1
1,Swiss Federal Institute of Technology of Lausanne,1
2,University of Alabama at Birmingham,1
3,Aarhus University,1
4,Ben-Gurion University of the Negev,1
...,...,...
652,University of Duesseldorf,1
653,Linkoping University,1
654,University of Marburg,1
655,Rensselaer Polytechnic Institute,1


In [57]:
tc = pd.merge(times2, cwur2, on='university_name', how='outer')

In [58]:
tcs = pd.merge(tc, shan2, on='university_name', how='outer').replace(np.NaN, 0)

In [59]:
tcs['somma'] = tcs['count_times'] + tcs['count_cwur'] + tcs['count_shan']

In [60]:
tcs[['university_name', 'somma']]

,university_name,somma
0,National Taiwan University,3.0
1,Hunan University,2.0
2,National Institute of Applied Sciences of Lyon,1.0
3,University of Jyväskylä,2.0
4,Bayreuth University,1.0
...,...,...
1426,The University of Reading,1.0
1427,College of France,1.0
1428,University of Arkansas at Fayetteville,1.0
1429,University of Duesseldorf,1.0


## 11. In the times ranking, compute the number of times each university appears

Raggruppiamo le università del dataset times e successivamente andiamo a contare le righe e quindi il numero di volte in cui è presente.

In [78]:
times_count = times.groupby('university_name').count()['world_rank'].reset_index().rename(columns={"world_rank": "count_times"})
times_count

,university_name,count_times
0,AGH University of Science and Technology,1
1,Aalborg University,5
2,Aalto University,5
3,Aarhus University,6
4,Aberystwyth University,5
...,...,...
813,École Normale Supérieure,6
814,École Normale Supérieure de Lyon,6
815,École Polytechnique,6
816,École Polytechnique Fédérale de Lausanne,6


## 12. Find the universities that appear at most twice in the times ranking

In [62]:
times12 = times.groupby('university_name').size()

In [63]:
times12[times.groupby('university_name').size()<3]

university_name
AGH University of Science and Technology    1
Adam Mickiewicz University                  1
Aix-Marseille University                    1
Ajou University                             1
Alexandru Ioan Cuza University              1
                                           ..
Yokohama City University                    1
Yokohama National University                1
Yuan Ze University                          2
Yıldız Technical University                 1
Örebro University                           1
Length: 408, dtype: int64

## 13. The universities that, in any year, have the same position in all three rankings (they must have the same position in a year).

Per andare a verificare se un0università ha la stessa posizione in tutte e tre le classifiche in un dato anno possiamo semplicemente fare due join considerando come chiave il nome dell'università e l'anno. In questo modo avremo l'intersezione delle università presenti in tutte le classidfiche nello stesso anno. Successivamente confrontiamo le varie colonne pr prendere solo le università con lo stesso rank. Dal momento che alcuni rank nel times dataset e nel shanghai dataset consideriamo, come nell'esercizio 4, il valore medio applicando la funzione "pulisci" dell'esercizio 4. Così facendo vediamo che l'unica università con la stessa posizione nello stesso anno è l'università di Stanford.

In [65]:
times['world_rank'] = times["world_rank"].apply(pulisci)
shan['world_rank'] = shan['world_rank'].apply(pulisci)

In [66]:
merge = pd.merge(times[['world_rank','university_name','year']],shan[['world_rank','university_name','year']], on=['university_name','year'],
                suffixes=["_times", "_shan"])

In [67]:
merge_tot = pd.merge(merge, cwur[['world_rank','university_name','year']], on=['university_name','year'])

In [73]:
merge_tot2 = merge_tot[merge_tot['world_rank_times'] == merge_tot['world_rank_shan']][['university_name','world_rank_times','world_rank_shan','world_rank','year']]

In [74]:
merge_tot2[merge_tot2['world_rank_times'] == merge_tot2['world_rank']]

,university_name,world_rank_times,world_rank_shan,world_rank,year
61,Stanford University,2,2,2,2013


### Alternativa: considero un'università nella stessa posizione se gli intervalli dei rank si intersecano e il valore di cwur è nell'intersezione degli intervalli

Dal momento che molte università nei dataset times e shanghai sono classificate in un intervallo possiamo considerare una condizione più balanda di posizione uguale, ovvero diremo che un'unniversità è nella stessa posizione nelle tre classifiche se 
* le università hanno lo stesso rank;
* gli intervalli si intersecano e il rank di cwur è nell'intersezione;
* i due classifiche hanno lo stesso rank e questo è contenuto nell'intervallo della terza classifica.

Per fare ciò abbiamo creato la funzione "confronta" che considera tutte le possibili combinazioni e ritorna True se le università sono nella stessa posizione, altrimenti ritorna False.

In [70]:
def confronta(x, y, z):
    re_comp1 = re.compile("^(\d+)\-+(\d*)")
    #caso 1: times e shan intervalli
    m = re_comp1.search(x)
    m2 = re_comp1.search(y)
    if m and m2:
        if max(int(m.group(1)), int(m2.group(1))) < min(int(m.group(2)), int(m2.group(2)))\
        and z in range(max(int(m.group(1)), int(m2.group(1))), min(int(m.group(2)), int(m2.group(2)))):
            return True
        return False

    #caso2: times intervallo, shan no
    if m and not m2:
        if int(m.group(1)) < int(y) < int(m.group(2)) and int(y) == int(z):
            return True
        return False

    #caso3: shan intervallo, times no
    if not m and m2:
        if int(m2.group(1)) < int(x) < int(m2.group(2)) and int(x) == int(z):
            return True
        return False
    
    #caso 4: tutti numeri
    if int(x) == int(y) == int(z):
        return True
    return False

In [71]:
def togli_uguale(x):
    re_comp = re.compile("^\=(\d*)$")
    m = re_comp.search(x)
    if m:
        return m.group(1)
    return x

Per prima cosa eliminaimo gli "=" da dataset con la funzione "togli_uguale", successivamente faremo sempre i due join per ottenere i rank di tutte le classifiche. A questo dataset (merge_tot) aggiungiamo una colonna applicando la funzione "confonrta", infine filtraimo solo quelle che hanno valore True.

In [72]:
times['world_rank'] = times['world_rank'].apply(togli_uguale)

TypeError: expected string or bytes-like object

In [ ]:
shan['world_rank'] = shan['world_rank'].apply(togli_uguale)

In [ ]:
merge = pd.merge(times[['world_rank','university_name','year']],shan[['world_rank','university_name','year']], on=['university_name','year'],
                suffixes=["_times", "_shan"])

In [ ]:
merge_tot = pd.merge(merge, cwur[['world_rank','university_name','year']], on=['university_name','year'])

In [ ]:
merge_tot['uguale?'] = merge_tot[['world_rank_times', 'world_rank_shan', 'world_rank']]\
.apply(lambda x : confronta(x['world_rank_times'], x['world_rank_shan'], x['world_rank']), axis=1)

In [ ]:
merge_tot[merge_tot['uguale?'] == True]